In [ ]:
from jupyter_imports import *
import copy

In [ ]:
original_pnl = pd.read_csv(os.path.join(os.sep, 'tmp','pfoptimizer','pnl.csv'), 
                           date_parser=parser.parse, 
                           parse_dates=['end_time'],
                           index_col=0)
original_trajectory = pd.read_csv(os.path.join(os.sep, 'tmp','pfoptimizer','trajectory.csv'))
equity = 1e7

In [ ]:
end = datetime.now().replace(tzinfo=timezone.utc)
start = end - timedelta(days = 365)
pnl = original_pnl[(original_pnl['end_time']>start)&(original_pnl['end_time']<end)]
pnl['amtUSD'] *= 1e4/equity # all in bps
# correct error(temporary !)
pnl.loc[pnl['bucket']=='carry(USD not annualized)','amtUSD'] = pnl.loc[pnl['bucket']=='carry(USD not annualized)','amtUSD'] / 8.

carry = pnl[pnl['bucket']=='carry(USD not annualized)'].drop(columns='bucket')
weights = pnl[pnl['bucket']=='weights'].drop(columns='bucket')
IR01 = pnl[pnl['bucket']=='IR01(USD)'].drop(columns='bucket')

# Excess return in bps
## assumes spreads are done at mid every funding period
## Pls note there is no hyperparamters (so no overfit here)
### * carry = funding-borrow
### * IR01 = spot*premium chg
### * delta = 0 (only exists during exec)

In [ ]:
pnl[(pnl['name']=='total')&(pnl['bucket']!='weights')].pivot_table(index='end_time',columns='bucket',values='amtUSD').cumsum().iplot()

In [ ]:
pnl.pivot_table(index='end_time',columns='bucket',values='amtUSD').iplot(secondary_y='weights')

## weights

In [ ]:
carry.pivot_table(index='end_time', columns='name').xs('amtUSD', axis=1).cumsum().iplot(secondary_y=['USDT', 'total'])

In [ ]:
avg_weights = weights.pivot_table(index='end_time', columns='name').mean().xs('amtUSD').drop(['total', 'USDT'])/1e4
print(f'leverage = {avg_weights.sum()}')
funding = original_trajectory.pivot_table(index='time', columns='name', values='funding')
averages = pd.DataFrame({'avg_weight_pct': avg_weights*100,
            'avg_funding_pct': funding.mean()*100}).sort_values('avg_weight_pct', key=abs, ascending=False)
plt.scatter(x=averages['avg_funding_pct'], y=averages['avg_weight_pct'])
averages.head(20)

# tracking error

In [ ]:
df = copy.deepcopy(original_trajectory)
df = df.loc[df['name']=='total', ['time', 'RealizedCarry', 'ExpectedCarry']].set_index('time')
df['tracking_error'] = (df['RealizedCarry']-df['ExpectedCarry'])
df = df.iloc[24:]/equity
df.iplot()

# benchmarks

In [ ]:
df = copy.deepcopy(original_trajectory)
df = df[df['name']!='USDT']
df['RealizedCarry'] /= equity
df['carry'] = df.apply(lambda x: max(-x['funding']-x['borrow'],x['funding']), axis=1)
df['carry'] = df['funding']
pivot = df.pivot_table(index='time', columns='name')
benchmarks = pd.concat([pivot.xs(['carry'], axis=1), pivot[[('RealizedCarry', 'total')]]], axis=1).droplevel(0, axis=1)
stats = pd.DataFrame()
stats['vs mean'] = (benchmarks['total']-benchmarks.drop(columns='total').mean(axis=1))
stats['vs median'] = (benchmarks['total']-benchmarks.drop(columns='total').median(axis=1))
stats['vs 1sigma'] = stats['vs mean']-benchmarks.drop(columns='total').std(axis=1)
stats['vs BTC'] = (benchmarks['total']-benchmarks['BTCUSDT'])
stats['vs ETH'] = (benchmarks['total']-benchmarks['ETHUSDT'])
stats['vs ARB'] = (benchmarks['total']-benchmarks['ARBUSDT'])
stats.expanding().mean().iloc[24:].iplot()